In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

In [ ]:
df = pd.read_csv(r"C:\Users\User\OneDrive\Desktop\Data_Science\Uptime\Python_Scripts\science_water_revenue_paradox\data\analysis_dataset.csv")

In [3]:
# =========================================================
# 1) Ensure year numeric + (optional) filter years 2021–2024
# =========================================================
df = df.copy()
df["year"] = pd.to_numeric(df["year"], errors="coerce")
df = df[(df["year"] >= 2021) & (df["year"] <= 2024)].copy()

# =========================================================
# 2) Build comparable metric per_person_per_year
#    IMPORTANT FIX:
#    Exclude invalid denominators (wp_pop_last <= 0) so we don't
#    keep +inf values (Python) that Excel drops as invalid.
# =========================================================
df = df[pd.to_numeric(df["wp_pop_last"], errors="coerce") > 0].copy()
df["wp_revenue_USD"] = pd.to_numeric(df["wp_revenue_USD"], errors="coerce")
df["wp_pop_last"] = pd.to_numeric(df["wp_pop_last"], errors="coerce")

df["per_person_per_year"] = df["wp_revenue_USD"] / df["wp_pop_last"]

# Keep finite values only (safety)
df = df[np.isfinite(df["per_person_per_year"])].copy()

# Base set: non-negative (includes zeros), aligns with Excel
df_filtered = df[df["per_person_per_year"] >= 0].copy()

# =========================================================
# 3) OUTLIER REMOVAL using IQR (on the base set)
# =========================================================
Q1 = df_filtered["per_person_per_year"].quantile(0.25)
Q3 = df_filtered["per_person_per_year"].quantile(0.75)
IQR = Q3 - Q1

lower = Q1 - 1.5 * IQR
upper = Q3 + 1.5 * IQR

filtered_df = df_filtered[
    (df_filtered["per_person_per_year"] >= lower) &
    (df_filtered["per_person_per_year"] <= upper)
].copy()

# Remove zeros from the IQR-filtered set
filtered_non_zero_df = filtered_df[filtered_df["per_person_per_year"] > 0].copy()

# =========================================================
# 4) Stats helper
# =========================================================
def stats_summary(series: pd.Series):
    s = series.astype(float)
    return {
        "min": float(s.min()),
        "q1": float(s.quantile(0.25)),
        "median": float(s.median()),
        "q3": float(s.quantile(0.75)),
        "n": int(s.shape[0]),
    }

stats1 = stats_summary(df_filtered["per_person_per_year"])          # with outliers
stats2 = stats_summary(filtered_df["per_person_per_year"])          # without outliers
stats3 = stats_summary(filtered_non_zero_df["per_person_per_year"]) # no outliers & no zero

# (Optional) sanity prints
print("df.shape:", df.shape)
print("With outliers:", len(df_filtered))
print("Without outliers:", len(filtered_df))
print("Without outliers & zeros:", len(filtered_non_zero_df))
print("Zero values in df_filtered:", (df_filtered["per_person_per_year"] == 0).sum())
print("Zero values in filtered_df:", (filtered_df["per_person_per_year"] == 0).sum())

# =========================================================
# 5) PLOT
# =========================================================
fig = make_subplots(
    rows=1, cols=3,
    subplot_titles=("With Outliers", "Without Outliers", "Without Outliers & Zero Values")
)

color_main = "#0099a5"

# Main boxplots
fig.add_trace(go.Box(
    x=df_filtered["per_person_per_year"],
    name="",
    boxpoints="all",
    marker=dict(color=color_main, size=3)
), row=1, col=1)

fig.add_trace(go.Box(
    x=filtered_df["per_person_per_year"],
    name="",
    boxpoints="all",
    marker=dict(color=color_main, size=3)
), row=1, col=2)

fig.add_trace(go.Box(
    x=filtered_non_zero_df["per_person_per_year"],
    name="",
    boxpoints="all",
    marker=dict(color=color_main, size=3)
), row=1, col=3)

# Annotation boxes
positions = [0.22, 0.60, 0.98]
stats_list = [stats1, stats2, stats3]

for i, s in enumerate(stats_list):
    fig.add_annotation(
        x=positions[i], y=0.95, xref="paper", yref="paper",
        text=(
            f"Min: {s['min']:.3f}<br>"
            f"25%: {s['q1']:.3f}<br>"
            f"Median: {s['median']:.3f}<br>"
            f"75%: {s['q3']:.3f}<br>"
            f"N: {s['n']}"
        ),
        showarrow=False,
        font=dict(size=12, color="black"),
        align="left",
        bgcolor="#ebeaeb",
        bordercolor="#004d5c",
        borderwidth=2,
        borderpad=4,
        opacity=0.8
    )

fig.update_layout(
    title_text="Revenue per Person per Year (USD) - 2021 to 2024",
    template="plotly_white",
    height=600,
    width=1600,
    paper_bgcolor="lavender",
    plot_bgcolor="whitesmoke",
    title_font=dict(size=24, family="Arial", color="darkblue"),
    showlegend=False,
    margin=dict(l=50, r=50, t=100, b=50)
)

fig.update_xaxes(
    title_text="Per person per year revenue (USD)",
    title_font=dict(size=16, color="darkblue")
)
fig.update_yaxes(
    title_text="Water points",
    title_font=dict(size=16, color="darkblue")
)

fig.show()

pio.write_image(fig, "revenue_per_person.svg", format="svg")


df.shape: (11082, 45)
With outliers: 11082
Without outliers: 9847
Without outliers & zeros: 4872
Zero values in df_filtered: 4975
Zero values in filtered_df: 4975


In [4]:
# =========================================
# 1. Load SCHEMA dataset
# =========================================
df_grp = df[df["wp_type_cat"] == "Scheme"].copy()

# Ensure numeric year
df_grp["year"] = pd.to_numeric(df_grp["year"], errors="coerce")

# =========================================
# 2. Compute revenue per m³ (Scheme level)
# =========================================
df_grp = df_grp[(df_grp["wp_m3"] >= 1) & (df_grp["wp_revenue_USD"] >= 0)]
df_grp["revenue_per_m3"] = df_grp["wp_revenue_USD"] / df_grp["wp_m3"]
df_grp = df_grp[np.isfinite(df_grp["revenue_per_m3"])]

# =========================================
# 3. Filter positive values
# =========================================
df_filtered = df_grp.copy()

# =========================================
# 4. IQR OUTLIER REMOVAL
# =========================================
Q1 = df_filtered["revenue_per_m3"].quantile(0.25)
Q3 = df_filtered["revenue_per_m3"].quantile(0.75)
IQR = Q3 - Q1

lower = Q1 - 1.5 * IQR
upper = Q3 + 1.5 * IQR

filtered_df = df_filtered[
    (df_filtered["revenue_per_m3"] >= lower) &
    (df_filtered["revenue_per_m3"] <= upper)
]

filtered_non_zero_df = filtered_df[
    filtered_df["revenue_per_m3"] > 0
]

# =========================================
# 5. Stats summary function
# =========================================
def summarize(series):
    s = series.astype(float).values
    return {
        "min": float(np.min(s)),
        "p25": float(np.percentile(s, 25)),
        "median": float(np.median(s)),
        "p75": float(np.percentile(s, 75)),
        "max": float(np.max(s)),
        "mean": float(np.mean(s)),
        "std": float(np.std(s, ddof=1)),
        "n": int(len(s)),
    }

sum_with  = summarize(df_filtered["revenue_per_m3"])
sum_wo    = summarize(filtered_df["revenue_per_m3"])
sum_wo0   = summarize(filtered_non_zero_df["revenue_per_m3"])

# =========================================
# 6. Plot
# =========================================
color_main = "#47B8B8"

fig = make_subplots(
    rows=1, cols=3,
    subplot_titles=[
        "With Outliers",
        "Without Outliers",
        "Without Outliers & Zeros"
    ]
)

fig.add_trace(go.Box(
    x=df_filtered["revenue_per_m3"],
    boxpoints="all",
    marker=dict(color=color_main, size=3)
), row=1, col=1)

fig.add_trace(go.Box(
    x=filtered_df["revenue_per_m3"],
    boxpoints="all",
    marker=dict(color=color_main, size=3)
), row=1, col=2)

fig.add_trace(go.Box(
    x=filtered_non_zero_df["revenue_per_m3"],
    boxpoints="all",
    marker=dict(color=color_main, size=3)
), row=1, col=3)

# =========================================
# 7. Annotation boxes
# =========================================
positions = [0.22, 0.59, 0.98]
stats_list = [sum_with, sum_wo, sum_wo0]

def fmt(s):
    return (
        f"Min: {s['min']:.3f}<br>"
        f"25%: {s['p25']:.3f}<br>"
        f"Median: {s['median']:.3f}<br>"
        f"75%: {s['p75']:.3f}"
    )

for i, stat in enumerate(stats_list):
    fig.add_annotation(
        x=positions[i], y=0.95,
        xref="paper", yref="paper",
        text=fmt(stat),
        showarrow=False,
        align="left",
        font=dict(size=12, color="black"),
        bgcolor="#ebeaeb",
        bordercolor="#004d5c",
        borderwidth=2,
        borderpad=4,
        opacity=0.85
    )

# =========================================
# 8. Layout (WIDTH FIX APPLIED)
# =========================================
fig.update_layout(
    title_text="Revenue per m³ (USD) for Scheme Water Systems (2021–2024)",
    template="plotly_white",
    height=600,
    width=1600,          # ✅ critical fix
    paper_bgcolor="lavender",
    plot_bgcolor="whitesmoke",
    title_font=dict(size=24, family="Arial", color="darkblue"),
    showlegend=False,
    margin=dict(l=50, r=50, t=100, b=50)
)

fig.update_xaxes(
    title_text="Revenue per m³ (USD)",
    title_font=dict(size=16, color="darkblue")
)

fig.update_yaxes(
    title_text="Grouped Schemes",
    title_font=dict(size=16, color="darkblue")
)

# =========================================
# 9. Show & Export
# =========================================
fig.show()

pio.write_image(
    fig,
    "revenue_per_m3_scheme.svg",
    format="svg"
)

In [ ]:
# =========================================
# 1) Load Handpump Dataset
# =========================================
df = pd.read_csv(r"C:\Users\User\OneDrive\Desktop\Data_Science\Uptime\Python_Scripts\science_water_revenue_paradox\data\analysis_dataset.csv")
df_hp = df[df["wp_type_cat"] == "Handpump"].copy()

# Ensure numeric year
df_hp["year"] = pd.to_numeric(df_hp["year"], errors="coerce")

# =========================================
# 2) Filter Handpumps (2021–2024)
# =========================================
df_HP_africa = df_hp[
    (df_hp["wp_revenue_USD"] >= 0)
].copy()

# =========================================
# 3) Use Revenue per Handpump directly
# =========================================
df_HP_africa["revenue_per_handpump"] = df_HP_africa["wp_revenue_USD"]
df_HP_africa = df_HP_africa[np.isfinite(df_HP_africa["revenue_per_handpump"])]

# =========================================
# 4) IQR OUTLIER REMOVAL
# =========================================
Q1 = df_HP_africa["revenue_per_handpump"].quantile(0.25)
Q3 = df_HP_africa["revenue_per_handpump"].quantile(0.75)
IQR = Q3 - Q1

lower = Q1 - 1.5 * IQR
upper = Q3 + 1.5 * IQR

filtered_df_hp = df_HP_africa[
    (df_HP_africa["revenue_per_handpump"] >= lower) &
    (df_HP_africa["revenue_per_handpump"] <= upper)
]

filtered_non_zero_df_hp = filtered_df_hp[
    filtered_df_hp["revenue_per_handpump"] > 0
]

# =========================================
# 5) Stats Summary Helper
# =========================================
def stats_summary(series):
    return {
        "min": series.min(),
        "q1": series.quantile(0.25),
        "median": series.median(),
        "q3": series.quantile(0.75),
        "max": series.max(),
        "mean": series.mean(),
        "count": len(series)
    }

stats1 = stats_summary(df_HP_africa["revenue_per_handpump"])             # With outliers
stats2 = stats_summary(filtered_df_hp["revenue_per_handpump"])           # No outliers
stats3 = stats_summary(filtered_non_zero_df_hp["revenue_per_handpump"])  # No outliers & zeros

# =========================================
# 6) Plot
# =========================================
fig = make_subplots(
    rows=1, cols=3,
    subplot_titles=[
        "With Outliers",
        "Without Outliers",
        "Without Outliers & Zero Values"
    ]
)

color_main = "#47B8B8"

fig.add_trace(go.Box(
    x=df_HP_africa["revenue_per_handpump"],
    name="",
    boxpoints="all",
    marker=dict(color=color_main, size=4)
), row=1, col=1)

fig.add_trace(go.Box(
    x=filtered_df_hp["revenue_per_handpump"],
    name="",
    boxpoints="all",
    marker=dict(color=color_main, size=4)
), row=1, col=2)

fig.add_trace(go.Box(
    x=filtered_non_zero_df_hp["revenue_per_handpump"],
    name="",
    boxpoints="all",
    marker=dict(color=color_main, size=4)
), row=1, col=3)

# =========================================
# 7) Annotation Boxes
# =========================================
positions = [0.2, 0.59, 0.98]
stats_list = [stats1, stats2, stats3]

def fmt(s):
    return (
        f"Min: {s['min']:.2f}<br>"
        f"25%: {s['q1']:.2f}<br>"
        f"Median: {s['median']:.2f}<br>"
        f"75%: {s['q3']:.2f}"
    )

for i, s in enumerate(stats_list):
    fig.add_annotation(
        x=positions[i], y=0.95,
        xref="paper", yref="paper",
        text=fmt(s),
        showarrow=False,
        align="left",
        font=dict(size=12, color="black"),
        bgcolor="#ebeaeb",
        bordercolor="#004d5c",
        borderwidth=2,
        borderpad=4,
        opacity=0.8
    )

# =========================================
# 8) Layout (WIDTH FIX ADDED)
# =========================================
fig.update_layout(
    title="Revenue per Handpump per Year (2021–2024)",
    template="plotly_white",
    height=600,
    width=1600,   # ✅ keep wide layout in SVG export
    paper_bgcolor="lavender",
    plot_bgcolor="whitesmoke",
    title_font=dict(size=24, family="Arial", color="darkblue"),
    showlegend=False,
    margin=dict(l=50, r=50, t=100, b=50)
)

fig.update_xaxes(
    title_text="Revenue per Handpump per Year (USD)",
    title_font=dict(size=16, color="darkblue")
)

fig.update_yaxes(
    title_text="Handpump Water Points",
    title_font=dict(size=16, color="darkblue")
)

# =========================================
# 9) Show & Export (optional)
# =========================================
fig.show()

pio.write_image(fig, "revenue_per_handpump.svg", format="svg")